# Quiz Evaluator/Tutor Agent

This notebook demonstrates how to create a Quiz Evaluator/Tutor agent using Semantic Kernel.
The agent evaluates student answers against ground truth answers and provides insights into potential misunderstandings.

In [ ]:
# Install required packages if not already installed
# !pip install semantic-kernel>=1.3.0 nest-asyncio

## Setup Environment

Configure the environment variables for authentication with your AI service.

In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

import os
import asyncio
from IPython.display import Markdown, display

# Setup for Azure OpenAI
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Alternatively, for OpenAI
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Uncomment and configure one of the above sections based on your preferred AI service

## Import Libraries and Create Kernel

In [1]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import KernelArguments

# Choose the appropriate AI service connector
# For Azure OpenAI
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# For OpenAI
# from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# Create the kernel
kernel = Kernel()

# Add the appropriate AI service
# For Azure OpenAI
service_id = "quiz-evaluator"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
    )
)

# For OpenAI
# kernel.add_service(
#     OpenAIChatCompletion(
#         service_id=service_id,
#         ai_model_id="gpt-4-turbo",
#         api_key=os.environ["OPENAI_API_KEY"]
#     )
# )

NameError: name 'os' is not defined

## Create the Quiz Evaluator/Tutor Agent

We'll create an agent with specific instructions for evaluating quiz answers and providing tutoring feedback.

In [4]:
INSTRUCTIONS = """
You are an expert quiz evaluator and tutor. Your task is to evaluate a student's answer against 
a ground truth answer for a given quiz question.

Follow these steps in your evaluation:

1. Compare the student's answer with the ground truth answer for factual and conceptual accuracy.
2. Determine if the student's answer is correct, partially correct, or incorrect.
3. If the answer is incorrect or partially correct, identify specifically what concepts the student likely misunderstood.
4. Provide a detailed explanation of the misunderstanding and the correct understanding.
5. Use specific evidence from the student's answer to support your analysis.

Your response should be structured as follows:
- Evaluation: [Correct/Partially Correct/Incorrect]
- Analysis: [Brief analysis of the student's answer]
- Misunderstandings: [If applicable, what concepts were misunderstood]
- Explanation: [Clear explanation of the correct concepts]

Be precise, educational, and supportive in your feedback.
"""

# Create the quiz evaluator agent
quiz_evaluator_agent = ChatCompletionAgent(
    kernel=kernel,
    name="QuizEvaluatorTutor",
    instructions=INSTRUCTIONS,
)

## Create a Function to Evaluate Quiz Answers

In [5]:
async def evaluate_answer(question: str, ground_truth: str, student_answer: str) -> str:
    """Evaluate a student's quiz answer against the ground truth.
    
    Args:
        question: The quiz question
        ground_truth: The correct answer
        student_answer: The student's submitted answer
        
    Returns:
        The agent's evaluation and feedback
    """
    # Create a new chat history for this evaluation
    history = ChatHistory()
    
    # Format the prompt with all the necessary information
    prompt = f"""Please evaluate the following quiz response:
    
    Question: {question}
    
    Ground Truth Answer: {ground_truth}
    
    Student Answer: {student_answer}
    """
    
    # Add the prompt to the chat history
    history.add_message(ChatMessageContent(role=AuthorRole.USER, content=prompt))
    
    # Get the agent's response
    response = await quiz_evaluator_agent.get_response(history=history)
    
    return response.content

# Create a function that displays formatted markdown output
def display_evaluation(evaluation_text: str) -> None:
    display(Markdown(evaluation_text))

## Example Usage

In [6]:
# Example 1: Correctly answered question
question1 = "What is the law of conservation of energy?"
ground_truth1 = "The law of conservation of energy states that energy cannot be created or destroyed, only transferred or converted from one form to another. The total energy in an isolated system remains constant."
student_answer1 = "Energy can't be created or destroyed, just changed from one form to another. In a closed system, the total energy remains the same."

evaluation1 = await evaluate_answer(question1, ground_truth1, student_answer1)
display_evaluation(evaluation1)

In [ ]:
# Example 2: Partially correct answer
question2 = "Explain how photosynthesis works."
ground_truth2 = "Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose. The process uses carbon dioxide and water, releases oxygen as a byproduct, and can be summarized by the equation: 6CO₂ + 6H₂O + light energy → C₆H₁₂O₆ + 6O₂."
student_answer2 = "Photosynthesis is how plants make their food using sunlight. They take in carbon dioxide and release oxygen. It happens in the leaves where the chlorophyll is."

evaluation2 = await evaluate_answer(question2, ground_truth2, student_answer2)
display_evaluation(evaluation2)

In [ ]:
# Example 3: Incorrect answer
question3 = "What causes the seasons on Earth?"
ground_truth3 = "Seasons on Earth are caused by the tilt of Earth's rotational axis away or toward the sun as it travels through its year-long orbit. The Earth's axial tilt is approximately 23.4 degrees. This tilt causes different parts of Earth to receive different amounts of solar radiation at different times of the year, resulting in the seasons."
student_answer3 = "Seasons are caused by the Earth's elliptical orbit around the sun. When Earth is closer to the sun, it's summer, and when it's farther away, it's winter."

evaluation3 = await evaluate_answer(question3, ground_truth3, student_answer3)
display_evaluation(evaluation3)

## Interactive Quiz Evaluator Interface

In [ ]:
from ipywidgets import widgets
from IPython.display import clear_output
import nest_asyncio

# Apply nest_asyncio to allow asyncio.run() within Jupyter
nest_asyncio.apply()

# Create text areas for input
question_input = widgets.Textarea(
    value='',
    placeholder='Enter the quiz question here...',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='80px')
)

ground_truth_input = widgets.Textarea(
    value='',
    placeholder='Enter the correct answer here...',
    description='Ground Truth:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='150px')
)

student_answer_input = widgets.Textarea(
    value='',
    placeholder='Enter the student\'s answer here...',
    description='Student Answer:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='150px')
)

evaluate_button = widgets.Button(
    description='Evaluate Answer',
    disabled=False,
    button_style='primary', 
    tooltip='Click to evaluate the student\'s answer',
    icon='check'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        print("Evaluating answer...")
        
        question = question_input.value
        ground_truth = ground_truth_input.value
        student_answer = student_answer_input.value
        
        if not question or not ground_truth or not student_answer:
            print("Please fill in all fields before evaluating.")
            return
        
        # Create an async function to handle the evaluation
        async def evaluate_and_display():
            evaluation = await evaluate_answer(question, ground_truth, student_answer)
            clear_output()
            display(Markdown(evaluation))
        
        # Schedule the async function on the existing event loop
        loop = asyncio.get_event_loop()
        loop.create_task(evaluate_and_display())

evaluate_button.on_click(on_button_clicked)

# Display the interface
display(question_input, ground_truth_input, student_answer_input, evaluate_button, output)

## Advanced: Quiz Evaluator with Streaming Response

This version shows how to get streaming responses from the agent for a more interactive experience.

In [10]:
async def evaluate_answer_streaming(question: str, ground_truth: str, student_answer: str) -> None:
    """Evaluate a student's quiz answer against the ground truth with streaming output.
    
    Args:
        question: The quiz question
        ground_truth: The correct answer
        student_answer: The student's submitted answer
    """
    # Create a new chat history for this evaluation
    history = ChatHistory()
    
    # Format the prompt with all the necessary information
    prompt = f"""Please evaluate the following quiz response:
    
    Question: {question}
    
    Ground Truth Answer: {ground_truth}
    
    Student Answer: {student_answer}
    """
    
    # Add the prompt to the chat history
    history.add_message(ChatMessageContent(role=AuthorRole.USER, content=prompt))
    
    # Use streaming response
    response_text = ""
    
    # Get streaming response
    async for response_chunk in quiz_evaluator_agent.invoke(history=history):
        chunk_text = response_chunk.content
        response_text += chunk_text
        
        # Clear previous output and display accumulated text
        clear_output(wait=True)
        display(Markdown(response_text))
        
        # Small delay for better visualization of streaming
        await asyncio.sleep(0.01)

In [11]:
# Example with streaming output
question4 = "Explain Newton's First Law of Motion."
ground_truth4 = "Newton's First Law of Motion states that an object at rest will stay at rest, and an object in motion will stay in motion with the same speed and direction, unless acted upon by an unbalanced force. This property is also known as inertia."
student_answer4 = "Newton's First Law says that objects in motion stay in motion until something stops them. This is why we wear seatbelts in cars."

await evaluate_answer_streaming(question4, ground_truth4, student_answer4)

## Conclusion

This notebook demonstrated how to create a quiz evaluator/tutor agent using Semantic Kernel. The agent evaluates student answers against ground truth answers and provides detailed feedback about potential misunderstandings. 

You can extend this implementation by:
1. Adding more structured output formats
2. Incorporating subject-specific knowledge or textbook references
3. Creating a database of common misconceptions to improve feedback
4. Adding follow-up questions based on identified misunderstandings